In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: DiscretePMS – Discrete Parameter-Mapping Sonification

This notebook introduces and demonstrates usage of the DiscretePMS sonecule.
* The sonecule spawns synth for each data point in a data set
* The synth offers several parameters that can set at init time
* Specifically the parameters are:
  * amplitude
  * frequency
  * sharpness
  * spatial panning
  * attack time
  * duration of the event
  * release time
* In many custom situations, a custom synth will be created and passed on for individual sonifications, replacing the default.
* The mapping specifies how data channels shall control the individual parameters

In [ ]:
# headers and imports for the demo
import sonecules as sn
import sc3nb as scn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt
import time

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

def a2d(**kwargs):
    return kwargs 

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
sns.pairplot(data=penguins_df, hue="species", height=1.2);

In [ ]:
df = dataframes['penguins']
df.columns

## Usage Demo for the DiscretePMS Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.scoresyn import DiscretePMS, pms

The following code cell shows everything needed 
- to create the sonecule with data, 
- to clear the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

Usually we want to use a specific synths which we would define using the backend.
- In this case the default backend is sc3nb, so we can create it using scn.SynthDef
- Later sonecules will offer a well curated library of pre-packaged synths, so that it is rarely necessary to craft your own.
- The default synth (if none is provided is "cpmssyn", a 'continuous synth for PMSon'). 
- It is a pitched tone with added vibrato 
- It offers the continuous controls:

| parameter | range      | meaning                                         |
| --------- | ---------- | ----------------------------------------------- |
| freq      | [20,0.5sr] | frequency [Hz]                                  |
| amp       | [0, 1]     | amplitude                                       |
| sharp     | [0, 10]    | sharpness, the higher the more sharp the sound  |
| pan       | [-1, 1]    | spatial panning from left to right              |
| dur       | [0,...]    | duration in seconds                             |
| att       | [0,...]    | attack time (<dur)                              |
| rel       | [0,...]    | release time (<dur)                             |
| vibfreq   | [0,...]    | vibrato frequency [Hz]                          |
| vibir     | [0,...]    | relative vibrato intensity (dfreq = vibir*freq) |

As currently no default synths are implemented we have to do the work ourselves

In [ ]:
scn.SynthDef("sydpms", 
"""{ | out=0, freq=400, dur=0.4, att=0.001, rel=0.5, amp=0.1, vibfreq=0, vibir=0, sharp=0, pan=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibir*freq, add: freq);
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*(sharp+1), mul: amp), 40);
    var env = EnvGen.kr(Env.new([0,1,1,0], [att, dur-att-rel, rel]), doneAction:2);
    Out.ar(out, Pan2.ar(sig, pan, env));
}""").add()

In [ ]:
# # demo for the syctick
scn.Synth("sydpms", a2d(freq=280, amp=0.1, att=0, dur=1, rel=0.1, sharp=5, vibfreq=5, vibir=0.02, pan=0))

The next cell shows all steps in specifying the mapping in one go

In [ ]:
df.columns

The mapping is just a dictionary where 
- keys are the parameters, and 
- values is either **a number** (for a constant) or **a dictionary** with some mandatory and some optional keys:
    - "col": column/feature of the dataset to be used 
    - "fn": mapping function(values, xr, yr), but strings such as "lin", "exp", "log"
        are allowed as shortcut
    - "yr": the target range for the parameter (implied from synths bounds if omitted)
    - "xr": the source range (which is implied from the data if omitted)
see documenation for further mapping flags


In [ ]:
# create a sonecule, initialized with the synth to be used
# second argument can provide defaults and bounds for parameters
sndpms = DiscretePMS("sydpms")

# the most flexible and raw definition as dictionary
test_mapping = {
    "onset" : {"col": "bill_length_mm",    "fn": "lin", "yr": [0,8]},
    "freq" :  {"col": "flipper_length_mm", "fn": "exp", "yr": [pam.midi_to_cps(40), pam.midi_to_cps(110)]},
    "sharp" : {"col": "body_mass_g",       "fn": "lin", "yr": [1, 4]},
    "pan" : 0, 
    "att" : 0, 
    "dur" : 0.25, 
    "rel" : 0.15, 
    "amp" : 0.1,
}

# clear the timeline 
ctx.timeline.reset() 

# and render the sonification into the timeline
sndpms.schedule(df=df, mapping=test_mapping)

# finally start the realtime playback at a given rate
sndpms.start(rate=1)

Usually this can be done in a more condensed way, by
- omitting the defaults and bounds (later synths will come with good such values anyway)
- using the `(column, mapping_function, yr)` tuple instead of a value dict 
  - e.g. `['flipper_length_mm', 'exp', [440, 880]]`
- using the a2d() function for the outer dict, which allows to write a dict as function kwargs
- using the **parse mapping specification** function `pms()` for the inner (d.h. value) dicts,
- starting the sonification by daisy-chaining of the start() method
- by removing the constructor for subsequent calls, i.e. reusing the sonecule sndpms so that only `sndpms.schedule().start()` is called

We apply most of these tipps, but for the mapping only condense by using the value tuple 

In [ ]:
sndpms = DiscretePMS("sydpms")
ctx.timeline.reset() 
sndpms.schedule(df=df, mapping={
    "onset" : ("bill_length_mm",    "lin", [0, 2]),
    "freq" :  ("flipper_length_mm", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    "sharp" : ("body_mass_g",       "lin", [1, 4]),
    "pan" : 0, "att" : 0, "dur" : 0.25, "rel" : 0.15, "amp" : 0.1,
}).start(rate=1)

No let's see how a2d turns the inner dictionary into argument, allowing to get rid of many quotes and enhance readability

In [ ]:
ctx.timeline.reset() 
sndpms.schedule(df=df, mapping=a2d(
    onset = ("bill_length_mm",    "lin", [0, 2]),
    freq  = ("flipper_length_mm", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp = ("body_mass_g",       "lin", [1, 4]),
    pan = 0, att = 0, dur = 0.25, rel = 0.15, amp = 0.1
    )).start(rate=1)

Finally let's use pms (parse mapping specification), which beautifies the parameter specification dictionary and flexibly allows to add features.
Note that in the following example 
- we specify the source range (xr) for the flipper_length range to xr=[190, 195]
- this is mapped to frequency and due to the clipping, we hear how many penguins
  - have lower than 190 mm flipper lengths (200 Hz tone)
  - have higher than 195 mm flipper length (400 Hz tone) 

In [ ]:
ctx.timeline.reset() 
sndpms.schedule(df=df, mapping=a2d(
    onset = pms("bill_length_mm",    "lin", [0, 6]),
    freq  = pms("flipper_length_mm", "exp", [200, 400], xr=[190, 195], clip="minmax"),
    sharp = pms("body_mass_g",       "lin", [1, 8]),
    pan = 0, att = 0, dur = 0.05, rel = 0.15, amp = 0.1
    )).start(rate=1)

For the dictionary synonymous keys can be used, defaulting specs to the first
* `'col'`: to specify the name of the pandas series uses as data for mapping
  * equivalent keys are `'n', 'name', 'feature', 'feat', 'f'`
* `'fn'`: to specify the mapping function to map from data feature to parameter 
  * equivalent key is `'via'`
  * supported values are: `"lin", "exp", "log"` 
  * not yet supported: `"poly({{n}})"`
* `'yr'`: to specify the target range `[y1, y2]` - can be unsorted if needed
  * equivalent keys are 'to' and 'yrange'
* `'xr'`:  to specify the source (data) range as `[min, max]` tuple
  * equivalent keys are `'within'` and `'xrange'`
* `'xqr'`: to specify the source range in quantiles
  * equivalent keys are 'within_q' and 'xqrange'
  * not yet supported
* `'clip'` to specify how mapping results are clipped
  * values are `"min", "max", "minmax", "" or None` (the latter: TBC)
  * ToDo: only minmax supported so far
* `'pre'` to specify one or many functions on the series to be performed prior to mapping
  * values are either strings such as `midicps, cpsmidi, ampdb, dbamp, floor, diff` (and soon a few more)
  * value can also be a list of such strings: execution is in order
  * instead of strings, functions (names or lambda expressions) can be specified
* `'post'` to specify one or many functions to modify the resulting series after mapping
  * see notes for `'pre'` for syntax
  * as example, 
    * instead of `fn="exp", yr=[pam.midicps(40), pam.midicps(60)]`
    * you could use `fn="lin", yr=[40, 60], post="midicps"`

In [ ]:
# create a sonecule, initialized with your data selection
sncpms = DiscretePMS("sydpms")
ctx.timeline.reset() 

# Variante 1: map_function:
# pms would return a dictionary, depending on call arguments, shortcuts defined for arguments such as fn
sncpms.schedule(df=df, mapping=a2d(
    onset = pms("bill_length_mm",    "lin", yr=[0, 12]),
    freq =  pms("flipper_length_mm", "exp", yr=[pam.midicps(30), pam.midicps(94)]),
    sharp = pms("body_mass_g",       "lin", yr=[1, 15]),
    pan =   pms("bill_depth_mm",     "lin", xr=[15, 16], yr=[-1, 1]), 
    att = 0.3,  dur = 1.0, rel = 0.8, amp = 0.3,
    )).start()

In [ ]:
# Variante 2: map_function with a2d (to be more pythonic)
# pms would return a dictionary, depending on call arguments
sncpms.schedule(df=df, mapping=a2d(
    onset       = pms("INDEX", "lin", yr=[0, 5]),
    freq        = pms("flipper_length_mm", "exp", yr=[pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp       = pms("body_mass_g", "lin", yr=[1, 4]),
    pan         = pms("bill_depth_mm", xr=[13, 30], fn="lin", yr=[-1, 1]), 
    att=0, dur=0.02, amp=0.1, rel=0
    )).start()

In [ ]:
# Variante 3: map_function with a2d (to be more pythonic)
# pms args would be taken as col, xr=None, fn="lin", yr=None (in this order) 
sncpms = DiscretePMS("sydpms")
ctx.timeline.reset() 
sncpms.schedule(df=df, mapping=a2d(
    onset       = pms("body_mass_g", fn="lin", yr=[2, 8]),
    freq        = pms(col="flipper_length_mm", fn="exp", yr=[pam.midi_to_cps(20), pam.midi_to_cps(70)]),
    sharp       = pms(col="body_mass_g", fn="lin", yr=[1, 4]),
    pan         = pms(col="bill_depth_mm", xqr=[0.2, 0.8], fn="lin", yr=[-1, 1]), 
    att=0, dur=0.05, amp=0.1, rel=0.25,
    )).start()

Note that the events remain in the timeline. 
Setting the 

In [ ]:
ctx.timeline

Setting the time actively to 0 will cue the playback to that onset and result in
a sonification to be replayed

In [ ]:
ctx.realtime_playback.time = 0

In [ ]:
# to free the timeline use
ctx.clear()

In [ ]:
# to stop all sound playing via the backend use 
ctx.stop()

In [ ]:
# note that the playback's latency is >0 - it can also be set
# but see mesonic for details and help
ctx.realtime_playback.processor.latency

In [ ]:
# for own mapping experiments, its useful to see all columns
df.columns

Using a2d we can write this more like code, removing clutter

In [ ]:
# as long as we reuse the synth, no need to create the object, but just use with different mappings
ctx.timeline.reset() 
sndpms.schedule(df, mapping=a2d(
    onset =     ("bill_length_mm", "lin", [0, 5]),
    freq =      ("body_mass_g", "exp", [pam.midi_to_cps(40), pam.midi_to_cps(110)]),
    sharp =     ("bill_depth_mm", "lin", [1, 2]),
    rel=        ("bill_length_mm", "lin", [0.3, 1.5]), 
    pan=0, att=0, dur=0.01, amp=0.1,
    )).start()

### One-To-Many Mapping: 

Example using a 1-channel ECG
- using the one-to-many mapping to give more saliency to  variations at different values (e.g., R-Peak, T-Wave, Iso-Electricity, Negative values)


In [ ]:
df = dataframes['ecg'].iloc[:, [0, 5]]# .to_frame()
df.plot(figsize=(10,2));

In [ ]:
# as long as we reuse the synth, no need to create the object, but just use with different mappings
sndpms = DiscretePMS("sydpms")
ctx.timeline.reset() 

In [ ]:
sndpms.schedule(df.iloc[:,:], mapping=a2d(
    onset =   pms("INDEX", "lin", [0, 10]),
    freq  =   pms(0, "lin", [60, 84], pre=["diff", "abs"], xr=[0, 0.3],
                        post=[ lambda x: np.round(x/5)*5, "midicps"]),
    amp =     pms(0, "lin", [-20, -5], pre="abs", xr=[0, 1], post="dbamp", clip="minmax"),
    sharp =   pms(0, "lin", [0, 2],      xr=[0,  0.2], clip="minmax"),
    rel =     pms(0, "exp", [0.01, 5],   xr=[0.25, 1], clip="minmax"),
    pan=0, att=0.01, dur=0,
    )).start()
sndpms.mapping_df.plot(subplots=True, figsize=(8,4))
ctx.timeline

In [ ]:
df.columns

In [ ]:
sndpms.schedule(df.iloc[:,:], mapping=a2d(
    onset =   pms(0, "lin", [0.05, 10]),
    freq  =   pms(5, "exp", [200, 1200]),
    amp =     pms(5, "exp", [0.05, 0.5], xr=[0.25, 1], clip="minmax"),
    sharp =   pms(5, "lin", [0, 2],      xr=[0,  0.2], clip="minmax"),
    rel =     pms(0, "exp", [0.02, 5],   xr=[0, 1], clip="minmax"),
    pan=0, att=0, dur=0,
    )).start()

df.plot(x=0, y=5, figsize=(3,3), lw=0.1, marker=".", ms=0.8);
# sndpms.mapping_df.plot(subplots=True, figsize=(8,4));


### Custom Mapping:

Mapping the 3 species to 3 instruments using buffers

### Hybrid Discrete Mappings

to be written

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# load your multi-channel data into an Asig, e.g. 
data = np.random.random((40, 4))-0.5 # 100 rows with 8 channels, here same fake data
data = np.cumsum(data,axis=0)
df = pd.DataFrame(data, columns=["c1", "c2", "c3", "c4"])
df.plot(subplots=True);
df.head()

In [ ]:
scn.SynthDef("syd0", 
"""{ | out=0, freq=400, dur=0.4, att=0.001, rel=0.5, amp=0.1, vibfreq=0, vibir=0, sharp=0, pan=0 | 
    var vib = SinOsc.ar(vibfreq, mul: vibir*freq, add: freq);
    var sig = HPF.ar(Formant.ar(vib, vib, bwfreq: vib*(sharp+1), mul: amp), 40);
    var env = EnvGen.kr(Env.new([0,1,1,0], [att, dur-att-rel, rel]), doneAction:2);
    OffsetOut.ar(out, Pan2.ar(sig, pan, env));
}""").add()

# load your data / select your data
mydf = df

# sonecule for your synth with defaults and bounds
sndpms = DiscretePMS("syd0")

# clear the timeline 
ctx.timeline.reset() 

# example custom function to quantize pitch mapping to grid of semitones
def myfreqfn(v, xr, yr, q=4):
    return pam.midi_to_cps(np.round(pam.linlin(v, *xr, *yr)/q)*q)

# and render the sonification into the timeline
sndpms.schedule(df=mydf, mapping=a2d(
    onset =     ("INDEX", "lin", [0, 3]),
    freq =      pms("c3", myfreqfn, [40, 70], q=7), # try q=7, q=12, q=1
    sharp =     ("c2", "lin", [0, 5]),
    dur =       ("c1", "exp", [0.05, 1]),
    rel =       ("c1", "exp", [0.05, 1]), 
    amp =       ("c4", "exp", [0.9, 0.01]),
    pan = 0, att = 0,
))

# finally start the realtime playback at a given rate
sncpms.start(rate=1)

# if needed: plot the timeline using 
ctx.timeline.plot()